In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from env.envs.hookScene import HookScene
from env.envs.mugScene import MugScene
from env.envs.spoonScene import SpoonScene
from scripts.hooking import get_reference as get_hooking_reference
from scripts.hanging import get_reference as get_hanging_reference
from scripts.scooping import get_reference as get_scooping_reference
from magic.match import match

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/yuyao/miniconda3/envs/magic/lib/python3.10/site-packages/diffusers/models/unet_2d_condition.py:20: FutureWarning: `UNet2DConditionOutput` is deprecated and will be removed in version 0.29. Importing `UNet2DConditionOutput` from `diffusers.models.unet_2d_condition` is deprecated and this will be removed in a future version. Please use `from diffusers.models.unets.unet_2d_condition import UNet2DConditionOutput`, instead.
  deprecate("UNet2DConditionOutput", "0.29", deprecation_message)
/home/yuyao/miniconda3/envs/magic/lib/python3.10/site-packages/diffusers/models/unet_2d_condition.py:25: FutureWarning: `UNet2DConditionModel` is deprecated and will be removed in version 0.29. Importing `UNet2DConditionModel` from `diffusers.models.unet_2d_condition` is deprecated and this will be removed in a future version. Please use `from diffusers.models.unets.unet_2d_condition import UNet2DConditionModel`, instead.
  deprecate("UNet2DConditionModel", "0.29", deprecation_message)


In [2]:
task_name = 'hooking'
tool_name = 'scissors'
mug_id = tool_name
spoon_id = tool_name
sd_dino = False
dift = True

In [3]:
if task_name == 'hooking':
    reference_img, reference_depth_img, reference_camera, reference_contact_center, reference_grasp_center, reference_scene, reference_init_pose, reference_pose = get_hooking_reference()
elif task_name == 'hanging':
    reference_img, reference_depth_img, reference_camera, reference_contact_center, reference_scene = get_hanging_reference()
    reference_grasp_center = None
elif task_name == 'scooping':
    plane_origin = np.array([0, -0.35, 0])
    plane_normal = np.array([0, -1, 0])
    (reference_img, reference_contact_center, reference_collide_center, reference_grasp_center,
     reference_pixel_to_3d_fn, reference_init_pose, reference_pcd_o3d) = get_scooping_reference(
        manual_center=False, plane_origin=plane_origin, plane_normal=plane_normal
    )
else:
    raise ValueError('Invalid task name')

[2024-10-11 12:34:37.662] [svulkan2] [error] GLFW error: X11: The DISPLAY environment variable is missing
[2024-10-11 12:34:37.662] [svulkan2] [warning] Continue without GLFW.
Found object data: name        hook
filename     NaN
scale       0.05
rotation     NaN
height       NaN
hook_pos     NaN
hook_rpy     NaN
Name: 0, dtype: object
Using additional scale: 0.025
PCD mean: [ 0.57458307 -0.54972685  0.20005789]
PCD min: [ 0.53719955 -0.66278052  0.18719953]


using default contact and grasp center (379, 200) (439, 532)


In [ ]:
if task_name == 'hooking':
    target_scene = HookScene(
        tool_name,
        'box',
    )

    target_scene.hide_env_visual()
    target_img, target_depth_img, target_camera = target_scene.get_picture(
        direction='+z',
        additional_translation=np.array([0.6, -0.6, 0.2-0.375*1.414]),
        debug_viewer=False,
        get_depth=True
    )
    target_scene.unhide_env_visual()
elif task_name == 'hanging':
    target_scene = MugScene(
        mug_id,
        add_robot=True,
        fps=480
    )

    target_scene.hide_env_visual()
    target_img, target_depth_img, target_camera = target_scene.get_picture(direction='+x', debug_viewer=False, get_depth=True)
    target_scene.unhide_env_visual()
else:
    plane_origin = np.array([0, -0.35, 0])
    plane_normal = np.array([0, -1, 0])
    target_scene = SpoonScene(
        spoon_id,
        fps=480,
        add_robot=True,
        radius=0.035
    )

    target_img, target_pixel_to_3d_fn = target_scene.get_slice(plane_origin, plane_normal)

results, resized_imgs, downsampled_images = match(reference_img, target_img, reference_contact_center, reference_grasp_center, only_compute_dino_feature=True, sd_dino=sd_dino, dift=dift)
results = results.permute(0, 2, 3, 1)

[2024-10-11 12:34:39.910] [SAPIEN] [warning] A second engine will share the same internal structures with the first one. Arguments passed to constructor will be ignored.
[2024-10-11 12:34:39.910] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.
[2024-10-11 12:34:39.910] [SAPIEN] [warning] Setting renderer more than once should be avoided.
Loading object from /home/yuyao/magic/utils/../assets/ig/processed_dataset/scissors/Diamond_Visions_Scissors_Red/Diamond_Visions_Scissors_Red_cm_vhacd.obj
Found object data: name        scissors
filename         NaN
scale            2.0
rotation     0,0,180
height           NaN
hook_pos         NaN
hook_rpy         NaN
Name: 24, dtype: object
Using additional scale: 1.0
Using additional rotation: [0.000000e+00 0.000000e+00 1.000000e+00 6.123234e-17]
Original rotation: [0.0, 0.0, 0.0, 1.0]
New rotation: [0.000000e+00 0.000000e+00 1.000000e+00 6.123234e-17]
P

diffusion_pytorch_model.safetensors:  15%|#5        | 524M/3.46G [00:00<?, ?B/s]

In [ ]:
import matplotlib 
matplotlib.use('TKAgg')
%matplotlib inline

fig, axes = plt.subplots(3, 5, figsize=(2 * resized_imgs[0].size[0] / 100, resized_imgs[0].size[1] / 100), dpi=100)
for i in range(3):
    for j in range(5):
        axes[i][j].axis('off')

axes[0][0].imshow(resized_imgs[0])
source_point = reference_contact_center
resized_source_point = np.array(source_point) / reference_img.size[1] * resized_imgs[0].size[0]
axes[0][0].scatter(resized_source_point[0], resized_source_point[1], c='r', s=50)


import cv2
from tqdm import tqdm 

for rotation_index in tqdm(range(0, 12)):
    downsampled_source_point = (np.array(source_point) / reference_img.size[1] * downsampled_images[0].size[1]).astype(int)
    source_feature = results[0][downsampled_source_point[1], downsampled_source_point[0]]

    target_feature = results[rotation_index+1]
    heatmap = torch.sum(target_feature * source_feature, dim=-1).cpu().numpy()

    # resize heatmap to target image size
    heatmap = cv2.resize(heatmap, (resized_imgs[0].size[0], resized_imgs[0].size[1]), interpolation=cv2.INTER_LINEAR)
    # gaussian filter
    heatmap = cv2.GaussianBlur(heatmap, (25, 25), 0)
    heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))

    # boost the heatmap where the feature is high for visualization
    heatmap = np.power(heatmap, 10)

    # get the coordinates of the maximum value in the heatmap
    max_index = np.unravel_index(heatmap.argmax(), heatmap.shape)
    target_point = max_index

    # set the size of the canvas to be the same as the img
    i = rotation_index // 4
    j = rotation_index % 4 + 1
    axes[i][j].imshow(resized_imgs[rotation_index+1])
    axes[i][j].imshow(heatmap, alpha=0.8)
    axes[i][j].scatter(target_point[1], target_point[0], c='r', s=50, marker='*')

plt.show()